# libs & configs

In [60]:

import pandas as pd
from pandas import to_datetime as pdDatetime
from datetime import datetime
from pandas import to_numeric as pdNumeric
import os
import sys
from collections import defaultdict


In [61]:
# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

In [62]:
from Scripts.functions import GetAno

# 00 - juntando das informações infosfot x Bling

## clientes °

In [89]:
# -- 1 lendo arquivos
df_clients_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_bling.xlsx")
df_clients_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_infosoft.xlsx")

# -- 2 - padroniza os nomes para maior facilidade
df_clients_infosoft.rename(columns={'Cgc Cpf':'Clie Cgc Cpf'}, 
                                inplace=True)
df_clients_bling.rename(columns={'CPF/CNPJ': 'Clie Cgc Cpf',
                                  'Nome':'Descrição', 
                                  'Endereço': 'Endereco',
                                  'UF':'Cd Uf', 
                                  'Cidade':'Nm Cidade', 
                                  'Bairro':'Bairro',
                                'Cliente desde':'Dt Cadastro', 
                                'Email':'E Mail'
                                }, inplace=True)

# -- 3 - faz o full join 
df_clientes_merge = pd.merge(df_clients_infosoft,df_clients_bling,
                                  on='Clie Cgc Cpf',   # chave primaria
                                  how='outer',    # full outer join no pandas       
                                  suffixes=('_infosoft','_bling')) # renomeias o sufixo das colunas (col1_df1,col2_df2...)

# -- 4 - filtrando para todos os clients ou seja '''full join com priorização do df1'''
cols = ['Descrição', 'Endereco', 'Cd Uf','Nm Cidade', 'Bairro', 'Dt Cadastro','E Mail'] # colunas que quero manter (na esquerda)
for col in cols: 
    df_clientes_merge[col] = df_clientes_merge[f'{col}_infosoft'].combine_first(df_clientes_merge[f'{col}_bling']) # where quando a esquerda for vazia colca conteudo da direita, colca tudo na nova coluna 
    
df_clientes_masquerade = df_clientes_merge[['Clie Cgc Cpf'] + cols] # atribui a df final o conteudo completo nas colunas novas sem esquecer da chave primeira



## nfs ~


In [90]:
'''
corrigir cod vendedores bling
nr de pedido

'''


# -- 1 - le os arquivos
df_nfs_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_bling.xlsx")
df_nfs_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_infosoft.xlsx")



# -- 2 - padroniza as colunas para evitar complicações
df_nfs_bling.rename(columns={'Data de emissão':'Dt. emissão', 
                             'Data de Saída/Entrada':'Dt Sai Ent', 
                             'Número':'Nr. nota', 
                             'CNPJ/CPF':'Clie Cgc Cpf',
                             'Código do vendedor':'Cd Vendedor', 
                             'Desconto':'Vl Total Desc', 
                             'Valor total líquido':'Vl Total Nota'
                             },inplace=True)

# -- 3 -  junta as duas df 
df_nfs_masquerade = pd.concat([df_nfs_bling,df_nfs_infosoft], axis=0)

# -- 4 - arrumando vendedores 

# -- 5 - padroniza os tipos
df_nfs_masquerade['Dt Sai Ent'] = pdDatetime(df_nfs_masquerade['Dt Sai Ent'])
df_nfs_masquerade['Dt. emissão'] = pdDatetime(df_nfs_masquerade['Dt. emissão'])
df_nfs_masquerade['Vl Total Desc'] = df_nfs_masquerade['Vl Total Desc'].str.replace(',','')
df_nfs_masquerade['Vl Total Nota'] = df_nfs_masquerade['Vl Total Nota'].str.replace(',','')
df_nfs_masquerade['Vl Total Desc'] = pdNumeric(df_nfs_masquerade['Vl Total Desc'],errors='coerce')
df_nfs_masquerade['Vl Total Nota'] =  pdNumeric(df_nfs_masquerade['Vl Total Nota'],errors='coerce')
df_nfs_masquerade['Nr Pedido'] =  pdNumeric(df_nfs_masquerade['Nr Pedido'],errors='coerce')

## pedidos

In [91]:
df_pedidos_masquerade = pd.read_excel(r"../Data/01-01-23_08-07-25/sh-pedidos_infosoft.xlsx")
df_pedidos_masquerade.rename(columns={'CNPJ - CPF':'Clie Cgc Cpf', 
                                      'Vl. total': 'Vl venda',
                                      'Nr. pedido': 'Nr Pedido'
                                      },inplace=True)

df_pedidos_masquerade['Nr Pedido'] = pdNumeric(df_pedidos_masquerade['Nr Pedido'])
df_pedidos_masquerade['Vl venda'] = df_pedidos_masquerade['Vl venda'].astype(str).str.replace(',','')
df_pedidos_masquerade['Vl venda'] = pdNumeric(df_pedidos_masquerade['Vl venda'])




## produtos

## vendedores °

In [92]:
df_vendedores_masquerade = pd.read_excel(r'..\Data\01-01-23_08-07-25\sh-vendedores_infosoft.xlsx')
df_vendedores_masquerade = df_vendedores_masquerade[~pd.isna(df_vendedores_masquerade['Nm Vendedor'])]
df_vendedores_masquerade['Cd Vendedor'] = df_vendedores_masquerade['Cd Vendedor'].astype(int) 

df_vendedores_masquerade

,aliquota,Nm supervisor,Cd supervisor,Cd Vendedor,Nm Vendedor,Endereco,Cd Uf,Ds Regiao,Bairro,Dt Cadastro,E Mail
1,7.0,ALDO,1.0,1,ALDO,NaN,NaN,NORTE,NaN,NaN,acz7@terra.com.br
4,7.0,FLAVIO E LILIAN,2.0,2,FLAVIO E LILIAN,NaN,NaN,NaN,NaN,43537.0,flavioelilian@hotmail.com
7,7.0,NaN,NaN,3,AV,NaN,NaN,NaN,NaN,43537.0,NaN
10,7.0,NaN,NaN,4,GERARDO,NaN,NaN,NaN,NaN,43537.0,NaN
13,7.0,NaN,NaN,5,ARLEY,NaN,NaN,NaN,NaN,43537.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
319,7.0,NaN,NaN,107,JOÃO PAULO RIO DE JANEIRO,NaN,NaN,NaN,NaN,45672.0,NaN
322,7.0,NaN,NaN,108,PAULO RIO DE JAINEIRO,NaN,NaN,NaN,NaN,45684.0,NaN
325,7.0,NaN,NaN,109,CAIO MARCELO VIGHI VISCARD E CIA LT,NaN,NaN,NaN,NaN,45833.0,NaN
328,7.0,ANDREIA,64.0,110,AMERIPEL CARLOS,NaN,NaN,NaN,NaN,45834.0,NaN



# 15 - faturamento completo mes ano, por "produto"



In [ ]:
''' mes / papelaria / mer_livre / sapatilha / eventos / modinha / puerilcultura / total 

total: / ....... /'''

##  16 - faturameno completo mes indivdual separado (modelo)



In [ ]:
'''
mes 
papelaria 
 mer_livre 
sapatilha 
eventos 
 modinha 
puerilcultura 
total 
'''

In [ ]:
# 1 preenchendo o modelo


# 05 - resumo de venda/faturamento - fechamento anual


In [ ]:
''' mes-ano / vendas / faturado 
total / .../ '''

Index(['Dt. emissão', 'Nr. nota', 'Nr Pedido', 'Clie Cgc Cpf', 'Cd Vendedor',
       'Vl Total Nota'],
      dtype='object')

In [102]:
# -- 1 - defines as bases de dados e trata certas colunas
df_nfs = df_nfs_masquerade[['Nr. nota','Dt. emissão', 'Nr Pedido', 'Vl Total Nota']]   
df_pedidos = df_pedidos_masquerade.copy()

# -- 2 -  defines as colunas do DataFrame
# colunas = ['mes-ano', 'vendas', 'faturamento']    
colunas = ['Nr. nota','Dt. emissão', 'Nr Pedido', 'Vl Total Nota']    
df_raw_mes_ano = pd.DataFrame(columns=colunas)

# -- 4 - coloca as informações no df princpal
df_raw_mes_ano = pd.concat([df_raw_mes_ano, df_nfs], axis=0)

# -- 5 - faz merge para epgar valor do pedido
df_raw_mes_ano = df_raw_mes_ano.merge(df_pedidos[['Nr Pedido', 'Vl venda']], 
                                                      on='Nr Pedido', how='left')

# -- 3 - pega os anos
anos = [2024, 2025]

# -- 4 - filtra com base nos anos e deixa penas o mes e o ano 
df_raw_mes_ano = df_raw_mes_ano[ # filtro
            (df_raw_mes_ano['Dt. emissão'].dt.year == anos[0]) |
            (df_raw_mes_ano['Dt. emissão'].dt.year == anos[1]) 
            ] 

df_raw_mes_ano['Dt. emissão'] = df_raw_mes_ano['Dt. emissão'].dt.to_period('M')

# -- 5 - atribui a noss df prinsipal 
df_faturamento_mes_ano = df_raw_mes_ano[['Dt. emissão', 'Vl venda','Vl Total Nota']]

# -- 6 - filtra por mes e soma venda e faturamento 
df_faturamento_mes_ano = df_faturamento_mes_ano.groupby('Dt. emissão')[['Vl venda','Vl Total Nota']].sum()
            

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10220\1684062161.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw_mes_ano = pd.concat([df_raw_mes_ano, df_nfs], axis=0)


In [104]:
df_faturamento_mes_ano.to_excel(r"..\reports\mes-ano.xlsx",index=False)

In [105]:
df_faturamento_mes_ano

,Vl venda,Vl Total Nota
Dt. emissão,,
2024-01,420757.22,441667.22
2024-02,165849.78,165449.78
2024-03,170551.65,170193.65
2024-04,271710.79,271023.82
2024-05,180781.29,184094.09
2024-06,381151.45,381450.16
2024-07,233164.85,233570.48
2024-08,430336.36,454616.36
2024-09,317589.82,319600.45


# 09 - comissões por periodos



In [57]:
''' 
supervisor
nm_vendedor
data / nf / cliente / venda / faturado / aliquota / comissão / cliente_nv / total
nm_vendedor / faturado
3% supervisor / subtotal
total / 
'''

' \nsupervisor\nnm_vendedor\ndata / nf / cliente / venda / faturado / aliquota / comissão / cliente_nv / total\nnm_vendedor / faturado\n3% supervisor / subtotal\ntotal / \n'

In [93]:
# -- 1 - defino as coluas de nossa df bruta e trago as df necessrias para o relatorio 
colunas = ['Clie Cgc Cpf','Dt. emissão', 'Nr. nota', 'Cd Vendedor', 'Vl Total Nota', 'aliquota', 'comissao', 'total']
df_comission_raw = pd.DataFrame(columns=colunas)
df_vendedores = df_vendedores_masquerade.copy() # para pegar o vendedor
df_clientes = df_clientes_masquerade.copy() # para pegar o cliente
df_pedidos = df_pedidos_masquerade.copy() # para pegar os pedidos
df_nfs = df_nfs_masquerade[['Dt. emissão', 'Nr. nota','Nr Pedido', 'Clie Cgc Cpf', 'Cd Vendedor', 'Vl Total Nota']]

# -- 1 - atribuindo infos das nfs 
df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)

# -- 2 - defininido o periodo (mes e ano) e filtrando nossa df
ano = 2025
mes = 6
df_comission_raw = df_comission_raw[(df_comission_raw['Dt. emissão'].dt.year == ano) & 
                (df_comission_raw['Dt. emissão'].dt.month == mes)]

# -- 3 - fazer um semi join (pega apegas nfs que tivrem com cd vendedor) - por preucação
df_comission_raw = df_comission_raw[df_comission_raw['Cd Vendedor'].isin(df_vendedores['Cd Vendedor'])] #  pandas não tem suporte direto para semi join o jeito a epanas filtrar oque e muito pratico!

# -- 4 - fazer merge para epgar o nome dos vendedores pelo cod, posteriormente pegar o supervisor e aliquota tambem 
df_comission_raw['aliquota'] = 7 # temporario
df_comission_raw = df_comission_raw.merge(df_vendedores[['Cd Vendedor', 'Nm Vendedor','Nm supervisor']], 
                             on='Cd Vendedor', how='left')

# -- 5 - fazer merge para pegar o nome do cliente (left join em duas colunas basicamente..)
df_comission_raw = df_comission_raw.merge(df_clientes[['Clie Cgc Cpf', 'Descrição']],
                      on='Clie Cgc Cpf', how='left')

# -- 6 - merge para pegar o valor do pedido
df_comission_raw = df_comission_raw.merge(df_pedidos[['Nr Pedido', 'Vl venda']],
                      on='Nr Pedido', how='left')

# -- 7 -  definindo comissão a ser recebida 
df_comission_raw['comissao'] = df_comission_raw['Vl Total Nota']*(df_comission_raw['aliquota']/100)

# -- 8 - definindo clientes novos
df_clientes_novos = df_clientes[df_clientes['Dt Cadastro'] > datetime(ano, mes, 1)]
df_clientes_novos['cliente_nv'] = 20

# -- 9 - fazendo merge para atribuir os clientes novos 
df_comission_raw = df_comission_raw.merge(df_clientes_novos[['Clie Cgc Cpf', 'cliente_nv']],
                      on='Clie Cgc Cpf', how='left', suffixes=('_raw','_new'))

# -- 10 - definindo comissão total 
df_comission_raw['total'] = df_comission_raw['cliente_nv'].fillna(0)+df_comission_raw['comissao'].fillna(0)

# --  - atribuindo informações a nossa df principal
df_comission = df_comission_raw[['Nm supervisor', 'Nm Vendedor','Dt. emissão', 'Nr. nota','Descrição','Vl venda', 'Vl Total Nota','aliquota','comissao','cliente_nv','total']]


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10220\543630500.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10220\543630500.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes_novos['cliente_nv'] = 20


In [59]:
df_comission.to_excel(r"..\reports\comissão-06-paga-07.xlsx",index=False)
